# First Practical Work

## Data Science and Engineering

### DESIGN OF AN IMAGE FILTER FUNCTION, PARALLELIZABLE AND SCALABLE

**Full name:** MARTA ALMAGRO FUELLO  
**NIA** : 100451979   


**Full name:** GRACIA ESTRAN BUYO    
**NIA** : 100452014

In [ ]:
import numpy as np
import multiprocessing as mp
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import cProfile
from multiprocessing.sharedctypes import Value, Array, RawArray
from multiprocessing import Process, Lock
import ctypes
import myfunctions as my

In [ ]:
F_IMAGE1 = "hand-x-ray.jpg" 
F_IMAGE2 = "fence.jpg" 
F_IMAGE3 = "chess.jpg" 
F_IMAGE4 = "digits.jpg" 
F_IMAGE5 = "cloudsonjupi.jpg" 

In [ ]:
image1 = np.array(Image.open(F_IMAGE1)) 
image2 = np.array(Image.open(F_IMAGE2)) 

In [ ]:
filter1 = np.array([
    [0,0,0,0,0],
    [0,0,0,0,0],
    [0,0,1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]])

filter2 = np.array([0.5, 0 , -0.5])

filter3 = np.array([[0.5],[0],[0.5]])  

filter4 = np.array([
    [1,0,-1],
    [2,0,-2],
    [1,0,-1]])

filter5 = np.array([
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.01330373,0.11098164,0.22508352,0.11098164,0.01330373],
    [0.00655965,0.05472157,0.11098164,0.05472157,0.00655965],
    [0.00078633,0.00655965,0.01330373,0.00655965,0.00078633]])


In [ ]:
NUMPROCESS = 4

In [ ]:
def tonumpyarray(mp_arr):
    #mp_array is a shared memory array with lock
    
    return np.frombuffer(mp_arr.get_obj(), dtype = np.uint8)

# AFTER THIS CELL YOU HAVE TO WRITE YOUR CODE

After this cell you have to use the preloaded image defined in the F_IMAGE variable, select two of the 5 filter predefined, and using the NUMPROCESS processors, apply the filters to the image and check results.

* The first filter is impulse response filter (the image output must be equals to the original one).
* The second filter is an edge filter, first order in x axis,  
* The third filter is an edge filter, first order in y axis,
* the fourth filter is an edge filter, second order, bi-directional
* the fifth filter is a blur gausian filter.

## First Part

### Define the variables for part 1:

We are going to prepare the data in order to parallelize the image filtering process. 
Fisrtly we declare the image and filter used and plot it to visualize how it is the original picture.


In [ ]:
image = np.array(Image.open(F_IMAGE1))
my_filter = filter3

In [ ]:
print('This is the original image')
plt.figure()
plt.imshow(image)

The function "image_filter" will receive the original image, the filter mask, the number of processors and a multiprocessing array. This last input is a shared memory with locks where the filtered image is going to be stored. 
The shared instance has to be created before running the function, because of that, the number of bytes in the image has to be calculated.

In [ ]:
data_buffer_size = image.shape[0] * image.shape[1] * image.shape[2] # Number of bytes

# Create the shared space with the right size
filtered_image_VECTOR = Array(ctypes.c_byte, data_buffer_size)

# Convert to an array and reshaped to a matrix
filtered_image = tonumpyarray(filtered_image_VECTOR).reshape(image.shape) 


The number of bytes is to create a c type array, that will be a vector and it will be used as shared space. Then it is converted to a NumPy variable (by using the function tonumpyarray) to use it with python syntax and reshaped to treat the image as a matrix, where the filtered image will be saved.

To avoid any race condition we create the lock instance and to start the parallelizing process we call the parallel_filter1. The method parallel_filter1 will have as goal to distribute the work between the processors and invoke the function that creates the global variables in a separated execution (init_globalimage1).


In [ ]:
def image_filter(image, filter_mask, numprocessors, filtered_image):

    lock = mp.Lock() # We create a lock instance
    my.parallel_filter(image, filtered_image, filter_mask, numprocessors, lock) 
    # We define the process    

With the needed shared memory matrix created, the function image_filter can be called.

In [ ]:
import importlib
importlib.reload(my)

image_filter(image, my_filter, NUMPROCESS, filtered_image_VECTOR)

In [ ]:
plt.figure()
plt.imshow(filtered_image)

## Second Part

On the other hand, in the second part there are two filters used therefore two memory spaces have to be established to use multiprocessor parallelized process.

In [ ]:
image = np.array(Image.open(F_IMAGE1))

data_buffer_size = image.shape[0] * image.shape[1] * image.shape[2] 

# Create both shared spaces with the right size
shared_space1 = Array(ctypes.c_byte, data_buffer_size) 
shared_space2 = Array(ctypes.c_byte, data_buffer_size)

filtered1_image_VECTOR = tonumpyarray(shared_space1).reshape(image.shape)
filtered2_image_VECTOR = tonumpyarray(shared_space2).reshape(image.shape)


The next function will invoke 2 different parallel processes and will use half
of the processors supplied to filter the image and store the two results in each shared memory definitions. The previous defined function will be called and will wait to both filters finished before return the control. 

In [ ]:
first_filter = filter3
second_filter = filter4

The method will divide by two the total number of processors in order to give one half to each process that filters the image with the correpondent filter. 
As in the last task we create the lock instance, but as this one is a multiprocessor process we also have to declare when they start and wait.

In [ ]:
def filters_execution(image, filter_mask1, filter_mask2,
                      numprocessors, filtered_image1, filtered_image2): 

    numprocess = numprocessors//2
    lock = mp.Lock()
    p1 = mp.Process(target = my.parallel_filter,
                 args = (image, filtered_image1, filter_mask1, numprocess, lock)) 
    p2 = mp.Process(target = my.parallel_filter,
                 args = (image, filtered_image2, filter_mask2, numprocess, lock))
    
    # Start both processes in parallel
    p1.start()
    p2.start()
    
    # Wait until both parallel tasks finish
    p1.join()
    p2.join()

In [ ]:
filters_execution(image, first_filter, second_filter, NUMPROCESS, shared_space1, shared_space2)

In [ ]:
plt.imshow(image)
plt.imshow(filtered1_image_VECTOR)

In [ ]:
plt.imshow(image)
plt.imshow(filtered2_image_VECTOR)

## Conclusions




We are distributing the task in the number of processors the computer has, in our case, 4, this smart way to use the processors makes the time of process to decrease. As the filtering problem does not imply that a cell has to wait to the previous cell (for example) to start the filtering process in the current one and we are able to make those process parallel and take advantage of the computer's processors. 

Furthermore, in the second part we are not only taking advantage of the processors we are also creating two parallel processes that perform different and have independent tasks, because each process have a different filter which means one of them could have a longer/bigger filter and will need more time. However the image will be the same and as it is independent task we could be filtering the image with the two filters at the same time, assigning to each process half of the processors that our computer has. Therefore each process will use in our case 2 processors.

Also as the instructions are telling us to keep it in a shared memory with locks so it does not run without control, destroying the previous data stored in the shared memory area, we placed where they are because there is where they can control correctly both parallel independent tasks.
